# Loading Dataset

Import dataset for google colab

In [ ]:
## Run these cells at the first time of running project
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
# use this cell when you upload the dataset in your colab
!unzip -q "/content/dataset.zip" -d "/content/"


In [ ]:
## use this cell whean you connect your drive and load dataset from drive
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/Colab Notebooks/dataset.zip" -d "/content/drive/MyDrive/dataset/"


Config dataset addresses


In [ ]:
img_train = "/content/drive/MyDrive/dataset/csvTrainImages 60k x 784.csv"
label_train = "/content/drive/MyDrive/dataset/csvTrainLabel 60k x 1.csv"
img_test = "/content/drive/MyDrive/dataset/csvTestImages 10k x 784.csv"
label_test = "/content/drive/MyDrive/dataset/csvTestLabel 10k x 1.csv"

Load Dataset

In [ ]:
import numpy as np

images_train = np.loadtxt(open(img_train), delimiter=',')
label_train = np.loadtxt(open(label_train), delimiter=',')
images_test = np.loadtxt(open(img_test), delimiter=',')
label_test = np.loadtxt(open(label_test), delimiter=',')

# train images count is 60K (60K train image and 60K their labels)
print("Size of data loaded for train images is: {}".format(images_train.shape))
print("Size of data loaded for train labels is: {}".format(label_train.shape))
# test image count is 10K (10K test image and 10K their labels)
print("Size of data loaded for test images is: {}".format(images_test.shape))
print("Size of data loaded for test labels is: {}".format(label_test.shape))


# Preprocessing

Presenting the raw samples without any preprocessing

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from numpy.random import randint

count_of_instances = 10 # set it to 6 to get the project appeal
_, axes = plt.subplots(nrows=1, ncols=count_of_instances,
                       figsize=(count_of_instances, 3))

for ax in axes:
    idx = randint(0, 60000) # choose a random element
    ax.set_axis_off()
    ax.imshow(images_train[idx].reshape(28, 28).T,
              cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Value: %i" % label_train[idx])

Normalizing

In [ ]:
import cv2

for item in range(images_train.shape[0]):
  images_train[item] = cv2.normalize(images_train[item], 
                                     None, 0, 1.0, 
                                     cv2.NORM_MINMAX, dtype=cv2.CV_32F).reshape((28*28,))

for item in range(images_test.shape[0]):
  images_test[item] = cv2.normalize(images_test[item], 
                                    None, 0, 1.0, 
                                    cv2.NORM_MINMAX, dtype=cv2.CV_32F).reshape((28*28,))


Thresholding

In [ ]:
import cv2
# train
for item in range(images_train.shape[0]):
  images_train[item] = cv2.threshold(images_train[item],
                                     0.2, 1.0, cv2.THRESH_BINARY)[1].reshape((28*28,))
  
# test
for item in range(images_test.shape[0]):
  images_test[item] = cv2.threshold(images_test[item], 
                                    0.2, 1.0, cv2.THRESH_BINARY)[1].reshape((28*28,))


Erosion & Dilation

In [ ]:
## Using Erosion on 0s and Dilation to all numbers to fix the
## broken vector caused by very large 0s and making the learning better.

import cv2
kernel = np.array([[0,1,1,1,0],
                   [0,1,1,1,0],
                   [1,1,1,1,1],
                   [0,1,1,1,0],
                   [0,1,1,1,0]], dtype=np.uint8)
# train
for item in range(images_train.shape[0]):
  if label_train[item] == 0:
    images_train[item] = cv2.erode(images_train[item].reshape((28,28)),
                                   kernel).reshape((784,))
    images_train[item] = cv2.erode(images_train[item].reshape((28,28)), 
                                   kernel).reshape((784,))

  images_train[item] = cv2.dilate(images_train[item].reshape((28,28)), 
                                  np.ones((3,3), np.uint8)  ).reshape((784,))
  images_train[item] = cv2.erode(images_train[item].reshape((28,28)), 
                                 np.ones((3,3), np.uint8)).reshape((784,))


# test
for item in range(images_test.shape[0]):
  if label_test[item] == 0:
    images_test[item] = cv2.erode(images_test[item].reshape((28,28)), 
                                  kernel).reshape((784,))
    images_test[item] = cv2.erode(images_test[item].reshape((28,28)), 
                                  kernel).reshape((784,))

  images_test[item] = cv2.dilate(images_test[item].reshape((28,28)), 
                                 np.ones((3,3), np.uint8)  ).reshape((784,))
  images_test[item] = cv2.erode(images_test[item].reshape((28,28)), 
                                np.ones((3,3), np.uint8)).reshape((784,))



Closing (Not Recommended)

In [ ]:
## this action may cause the accuracy to be lower than when it had not been applied
## you can skip running this cell

## Train
import cv2
kernel = np.array([[1,1,1,1],
                   [1,1,1,1],
                   [1,1,1,1],
                   [1,1,1,1],
                   ], dtype=np.uint8)

for item in range(images_train.shape[0]):
  images_train[item] = cv2.morphologyEx(images_train[item].reshape((28,28)), 
                                        cv2.MORPH_CLOSE, kernel).reshape((28*28,))

## test
for item in range(images_test.shape[0]):
    images_train[item] = cv2.morphologyEx(images_train[item].reshape((28,28)), 
                                          cv2.MORPH_CLOSE, kernel).reshape((28*28,))



Presenting Samples after preprocessing

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from numpy.random import randint

count_of_instances = 10 
_, axes = plt.subplots(nrows=1, ncols=count_of_instances,
                       figsize=(count_of_instances, 3))

for ax in axes:
    idx = randint(0, 60000) # choose a random element
    ax.set_axis_off()
    ax.imshow(images_train[idx].reshape(28, 28).T,
              cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Value: %i" % label_train[idx])

# Training the Model Using Various Parameters & Settings

### First Model

16/relu -> 16/relu -> 10/softmax | optimizer=SGD

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

model = keras.models.Sequential([
    layers.Dense(16, activation='relu', input_shape=(28*28,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# compile the model
model.compile(optimizer='sgd', 
              loss=keras.losses.SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

# train the model
model.fit(images_train, label_train, 
          epochs=10, batch_size=16, 
          validation_data=(images_test, label_test))

## FIT RESULT ##
## with raw dataset
#### loss: 1.8946 - accuracy: 0.1926 - val_loss: 1.9064 - val_accuracy: 0.1922
## with Erosion
#### loss: 1.6763 - accuracy: 0.2852 - val_loss: 1.6968 - val_accuracy: 0.2873
## with Erosion then Closing
#### loss: 1.9716 - accuracy: 0.2649 - val_loss: 1.9856 - val_accuracy: 0.2598
## with Thresholding then Erosion then Closing
## loss: 1.8743 - accuracy: 0.1994 - val_loss: 1.8705 - val_accuracy: 0.1933
## Nomal then Thresholding then Erosion then Closing
#### loss: 0.0459 - accuracy: 0.9872 - val_loss: 0.0695 - val_accuracy: 0.9792
## Nomal then Thresholding then Dilate+Erosion 
#### loss: 0.0566 - accuracy: 0.9844 - val_loss: 0.0930 - val_accuracy: 0.9740


Evaluate First Model

In [ ]:
# Evaluate 
test_loss, test_accuracy = model.evaluate(images_test, label_test)
print(test_loss, test_accuracy)

## Evaluate RESULT
## Raw Dataset
#### loss: 2.3026 - accuracy: 0.1000
## Erosion
#### loss: 1.6968 - accuracy: 0.2873
## Erosion ==> Closing
#### loss: 1.9856 - accuracy: 0.2598
## Thresholding ==> Erosion ==> Closing
#### loss: 1.8705 - accuracy: 0.1933
## Nomal ==> Thresholding ==> Erosion ==> Closing
#### loss: 0.0695 - accuracy: 0.9792
## Nomal ==> Thresholding ==> Dilate+Erosion 
#### loss: 0.0930 - accuracy: 0.9740

### Second Model

32/relu -> 32/relu -> 32/relu -> 10/softmax | Adam

In [ ]:
model = keras.models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(28*28,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# compile the model
model.compile(optimizer='adam', 
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# train the model
model.fit(images_train, label_train,
          epochs=8, batch_size=32,
          validation_data=(images_test, label_test))

## FIT RESULT ##
## raw dataset
#### loss: 0.0538 - accuracy: 0.9851 - val_loss: 0.1289 - val_accuracy: 0.9698
## Erosion
#### loss: 0.0458 - accuracy: 0.9880 - val_loss: 0.1012 - val_accuracy: 0.9761
## Erosion ==> Closing (kernel: 9*1)
#### loss: 0.0536 - accuracy: 0.9865 - val_loss: 0.1359 - val_accuracy: 0.9727
## Ersoion ==> dilation 
#### loss: 0.0447 - accuracy: 0.9888 - val_loss: 0.1137 - val_accuracy: 0.9773
## Thresholding ==> Erosion ==> Closing
#### loss: 0.0533 - accuracy: 0.9860 - val_loss: 0.1338 - val_accuracy: 0.9731
## Nomal ==> Thresholding ==> Erosion ==> Closing
#### loss: 0.0189 - accuracy: 0.9941 - val_loss: 0.0652 - val_accuracy: 0.9847
## Nomal ==> Thresholding ==> Dilate+Erosion 
#### loss: 0.0160 - accuracy: 0.9948 - val_loss: 0.0710 - val_accuracy: 0.9839

Evaluate second model

In [ ]:
# Evaluate 
test_loss, test_accuracy = model.evaluate(images_test, label_test)
print(test_loss, test_accuracy)

## Evaluate RESULT
## Raw Dataset
#### loss: 0.1289 - accuracy: 0.9698 
## Erosion
#### loss: 0.1012 - accuracy: 0.9761
## Erosion then Closing
#### loss: 0.1359 - accuracy: 0.9727
## Ersoion then dilation 
#### loss: 0.1137 - accuracy: 0.9773
## Thresholding ==> Erosion ==> Closing
#### loss: 1.8705 - accuracy: 0.1933
## Nomal ==> Thresholding ==> Erosion ==> Closing
#### loss: 0.0652 - accuracy: 0.9847
## Nomal ==> Thresholding ==> Dilate+Erosion 
#### loss: 0.0710 - accuracy: 0.9839

### Third model (best)

512/relu, 64/relu -> 10/softmax | Adam

In [ ]:
ACCURACY_THRESHOLD = 0.988
# ACCURACY_THRESHOLD = 0.995
class myCallback(keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') > ACCURACY_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!"\
                %(ACCURACY_THRESHOLD*100))   
          self.model.stop_training = True

callbacks = myCallback()


model = keras.Sequential([
    keras.layers.Dense(512, input_shape=(28*28, ), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(images_train, label_train, 
          epochs=10, batch_size=256, 
          validation_data=(images_test, label_test), 
          callbacks=[callbacks])


## FIT RESULT ##
## raw dataset
#### loss: 0.2492 - accuracy: 0.9392 - val_loss: 0.3228 - val_accuracy: 0.9265
## Erosion 3.22
#### loss: 0.0860 - accuracy: 0.9808 - val_loss: 0.1561 - val_accuracy: 0.9702
## Erosion ==> Closing
#### loss: 0.0854 - accuracy: 0.9816 - val_loss: 0.1923 - val_accuracy: 0.9636
## Thresholding ==> Erosion ==> Closing
####loss: 0.0878 - accuracy: 0.9784 - val_loss: 0.1747 - val_accuracy: 0.9669
## Thresholding ==> Erosion ==> Closing (32Neurons)
#### loss: 0.0795 - accuracy: 0.9859 - val_loss: 0.2930 - val_accuracy: 0.9750
## Nomal ==> Thresholding ==> Erosion ==> Closing
#### loss: 0.0177 - accuracy: 0.9942 - val_loss: 0.0886 - val_accuracy: 0.9820
## Nomal ==> Thresholding ==> Dilate+Erosion 
#### loss: 0.0113 - accuracy: 0.9963 - val_loss: 0.0999 - val_accuracy: 0.9811
## 3 Layer
#### loss: 6.0452e-04 - accuracy: 1.0000 - val_loss: 0.0541 - val_accuracy: 0.9880

Evaluate third model

In [ ]:
# Evaluate 
test_loss, test_accuracy = model.evaluate(images_test, label_test)
print(test_loss, test_accuracy)

## Evaluate RESULT
## Raw Dataset
#### loss: 0.3228 - accuracy: 0.9265 
## Erosion
#### loss: 0.1561 - accuracy: 0.9702
## Erosion then Closing
#### loss: 0.1923 - accuracy: 0.9636
## Thresholding ==> Erosion ==> Closing
#### loss: 0.1747 - accuracy: 0.9669
## Thresholding ==> Erosion ==> Closing (32Neurons)
#### loss: 0.2930 - accuracy: 0.9750
## Nomal ==> Thresholding ==> Erosion ==> Closing
#### loss: 0.0886 - accuracy: 0.9820
## Nomal ==> Thresholding ==> Dilate+Erosion 
## 3 Layer
#### loss: 0.0779 - accuracy: 0.9845

Showing images with their actual label & prediction

In [ ]:
from numpy.random import randint

predictions = model.predict(images_test)

predicted_labels = np.argmax(predictions, axis=1)


count_of_instances = 10 
_, axes = plt.subplots(nrows=1, ncols=count_of_instances, 
                       figsize=(count_of_instances, 3))

wrong_pred = np.where(predicted_labels != label_test)[0]
print(len(wrong_pred))

for ax in axes:
    idx = wrong_pred[randint(0, 120)] 
    ax.set_axis_off()
    ax.imshow(images_test[idx].reshape(28, 28).T, 
              cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("{}-> [{}]".format(int(label_train[idx]), 
                                    predicted_labels[idx]))